# Week 7 — Part 02: Config management + secrets (.env)

**Estimated time:** 60–90 minutes

## Learning Objectives

- Keep configuration out of code
- Use `.env` for local secrets without committing them
- Load environment variables early at startup
- Fail fast with clear error messages when secrets are missing


## Overview

Keep configuration out of code.

Never hardcode or commit API keys.

---

## Underlying theory: config is an input, secrets are a trust boundary

Treat configuration as runtime input that changes across environments:

- dev vs demo vs CI
- different datasets
- different models

Secrets are special because they grant access. So the boundary is:

- code can be public
- secrets must stay private

Practical implication:

- if you commit an API key once, assume it is leaked and rotate it
- `.env` is a local convenience, not a secure secret manager (but it’s fine for Level 1)

In [ ]:
import os
from pathlib import Path


try:
    from dotenv import load_dotenv
except Exception as e:  # pragma: no cover
    load_dotenv = None
    _dotenv_import_error = e


def load_env(path: str | None = None) -> None:
    if load_dotenv is None:
        raise RuntimeError(f"python-dotenv is required: {_dotenv_import_error}")
    if path:
        load_dotenv(dotenv_path=Path(path))
    else:
        load_dotenv()


def require_env(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise RuntimeError(f"Missing {name}. Put it in .env or set env var.")
    return value


print("env helpers ready")

In [ ]:
def build_config() -> dict:
    # TODO: build a config dict from env + defaults.
    # Example fields: model, timeout_s, max_retries.
    raise NotImplementedError


def redact_secret(secret: str, visible: int = 4) -> str:
    if len(secret) <= visible:
        return "*" * len(secret)
    return secret[:visible] + "*" * (len(secret) - visible)


print(redact_secret("sk-123456"))
print("Implement build_config().")

## Common pitfalls

- putting `.env` in git by accident (fix: ensure `.gitignore` includes it)
- mixing config and secrets in code defaults (fix: read from env, fail with a clear message)
- unclear error messages when secrets are missing (fix: say exactly which variable is required)

## References

- Twelve-Factor config: https://12factor.net/config
- python-dotenv: https://github.com/theskumar/python-dotenv